# Preprocessing
**Copyright 2023 (c) Naomi Chaix-Echel & Nicolas P Rougier**  
Released under a BSD 2-clauses license

This notebook reads and process the original dataset to ensure that tasks are named properly.
The original dataset is untouched and the processed dataset is saved using an alternative filename.

| Name           | Type     | Signification                 |
| :------------- | :------- | :---------------------------- |
| **subject_id** | string   | Identification of the subject |
| **date**       | datetime | Date whe then trial was made   | 
| **task_id**    | integer  | Identification of the task    | 
| **P_left**     | float    | Reward probability of the left stimulus |
| **V_left**     | float    | Reward amount of the left stimulus |
| **P_right**    | float    | Reward probability of the right stimulus |
| **P_right**    | float    | Reward amount of the right stimulus |
| **response**   | int      | Response (0: left, 1: right) |
| **reward**     | int      | Reward delivered (1) or not (0) |
| **RT**         | int      | Response time (ms) |


## Lottery description

For all the following types of lottery, we consider a choice between (x1, p1) and (x2, p2), xi being the value and pi being the probability:
* xi can be positive, null or negative: -3, -2, -1, 0, +1, +2, +3
* pi can be: 0.25, 0.50, 0.75 or 1.00


### Type 1 : x1 > 0 and x2 < 0, p1 = p2

* Lottery pairs containing one lottery with potential losses (LPL) and on lottery with potential gains (LPG)
* assess the discrimination of losses from the gains
* 36 different lottery pairs.

### Type 2 : p1 = p2 and x1 > x2 > 0

* LPG with a stochastic dominant option differentiating only by the x values
* assess the discrimination of positive x-values
* 12 different lottery pairs

### Type 3 : p1 = p2 and x1 < x2 < 0

* LPL with a stochastic dominant option differentiating only by the x values;
* assess the discrimination of negative x-values
* 12 different lottery pairs

### Type 4 : p1 > p2 and x1 = x2 > 0

* LPG with a stochastic dominant option differentiating only by the p values
* assess the discrimination of p-values associated to positive x-values
* 12 different lottery pairs 

### Type 5 : p1 < p2 and x1 = x2 < 0

* LPL with a stochastic dominant option differentiating only by the p values
* assess the discrimination of probabilities associated to negative quantities
* 18 different lottery pairs

### Type 6 : p1 < p2 and x1 > x2 > 0

* LPG with no stochastic dominant option
* 18 different lottery pairs.

### Type 7 : p1 < p2 and x1 < x2 < 0

* LPL with no stochastic dominant option
* 18 different lottery pairs.

## Import packages

In [1]:
import datetime                 # Time operations
import numpy as np              # Array operations
import pandas as pd             # Database operations
import matplotlib.pyplot as plt # Figures

## Load and Fusion data

In [2]:
print("Loading data... ", end="")

# Liste des noms de fichiers à charger et fusionner
files_to_concat = ["./data/ECO_2019-20.csv", "./data/ECO_2020-21.csv", "./data/ECO_2021-22.csv", "./data/ECO_2022-23.csv"]

# Charger chaque fichier CSV dans un DataFrame
dataframe = [pd.read_csv(filename, sep=";", decimal = ",") for filename in files_to_concat]

# Concaténer les lignes des DataFrames
original_data = pd.concat(dataframe, ignore_index=True)

# Sauvegarder le résultat dans un nouveau fichier CSV
original_data.to_csv("data.csv", index=False)

#Enlever les lignes des taches ECO_T (phase d'entrainement), Lot11 (tâches plus complexes) et ECO2 (affichage des options différent)
original_data = original_data[~original_data['tache'].str.contains("ECO_T|Lot11|ECO2")]

print("done!")

Loading data... done!


In [3]:
data_filename = "./data/data.csv"

## Filter, rename & retype fields

In [4]:
# Keep only relevant fields
data = original_data[["date_debut",
                      "nom_singe",
                      "tache",
                      "palier",
                      "r1_stim_gauche",
                      "r2_stim_gauche",
                      "p2_stim_gauche",
                      "r1_stim_droite",
                      "r2_stim_droite",
                      "p2_stim_droite",
                      "id_stim_gauche",
                      "id_stim_droite",
                      "stim_gauche_x",
                      "stim_gauche_y",
                      "stim_droite_x",
                      "stim_droite_y",
                      "stim_choisi",
                      "resultat",
                      "recompense",
                      "temps_demarrage",
                      "temps_reponse",
                      "id_module"]].copy()

# Rename fields
data = data.rename(columns={"nom_singe"      :    "subject_id",
                            "date_debut"     :    "date",
                            "tache"          :    "task_id",
                            "r1_stim_gauche" :    "V1_left",
                            "r2_stim_gauche" :    "V2_left",
                            "p2_stim_gauche" :    "P2_left",
                            "r1_stim_droite" :    "V1_right",
                            "r2_stim_droite" :    "V2_right",
                            "p2_stim_droite" :    "P2_right",
                            "temps_reponse"  :    "RT",
                            "temps_demarrage":    "ST"})

# Convertir les colonnes P2_left et P2_right en numériques
data['P2_left'] = pd.to_numeric(data['P2_left'], errors='coerce')
data['P2_right'] = pd.to_numeric(data['P2_right'], errors='coerce')

# Ajouter les colonnes P1_left et P1_right
data['P1_left'] = 1 - data['P2_left']
data['P1_right'] = 1 - data['P2_right']

data["EV_left"] = data["V1_left"] * data["P1_left"] + data["V2_left"] * data["P2_left"]
data["EV_right"] = data["V1_right"] * data["P1_right"] + data["V2_right"] * data["P2_right"]

# Convert date type (from string to datetime64)
data["date"] = pd.to_datetime(data["date"], dayfirst=True)

## Enrich data with actual gain or loss and subject's answer

In [5]:
# Ajouter la colonne "gain"
data['gain'] = data['recompense'].apply(lambda x: x if x > 0 else 0)

# Ajouter la colonne "loss"
data['loss'] = data['recompense'].apply(lambda x: x if x < 0 else 0)

# Ajouter la colonne "response" correspond au choix de l'option
data['response'] = data.apply(lambda row: 0 if row['stim_choisi'] == row['id_stim_gauche'] else (1 if row['stim_choisi'] == row['id_stim_droite'] else None), axis=1)


## Assign task id

In [6]:
# We assign task ids based on probablities and values
p1, x1 = data["P1_left"], data["V1_left"]
p2, x2 = data["P1_right"], data["V1_right"]

data.loc[(p1 == p2) & (x1 == x2), "task_id"] = 0

data.loc[(p1 == p2) & (x1 <  0) & (x2 >   0), "task_id"] = 1
data.loc[(p1 == p2) & (x2 <  0) & (x1 >   0), "task_id"] = 1

data.loc[(p1 == p2) & (x2 >  0) & (x1 >   x2), "task_id"] = 2
data.loc[(p1 == p2) & (x1 >  0) & (x2 >   x1), "task_id"] = 2

data.loc[(p1 == p2) & (x1 < x2) & (x2 < 0), "task_id"] = 3
data.loc[(p1 == p2) & (x2 < x1) & (x1 < 0), "task_id"] = 3

data.loc[(p1 >  p2) & (x1 >  0) & (x1 == x2), "task_id"] = 4
data.loc[(p2 >  p1) & (x1 >  0) & (x1 == x2), "task_id"] = 4

data.loc[(p1 >  p2) & (x1 <  0) & (x1 == x2), "task_id"] = 5
data.loc[(p2 >  p1) & (x1 <  0) & (x1 == x2), "task_id"] = 5

data.loc[(p1 <  p2) & (x1 > x2) & (x2 >   0), "task_id"] = 6
data.loc[(p2 <  p1) & (x2 > x1) & (x1 >   0), "task_id"] = 6

data.loc[(p1 <  p2) & (x1 < x2) & (x2 <   0), "task_id"] = 7
data.loc[(p2 <  p1) & (x2 < x1) & (x1 <   0), "task_id"] = 7

# Il reste des tâches non attribuées -> à définir

## Save new dataset

In [7]:
import os
filename, extension = os.path.splitext(data_filename)
filename = f"{filename}-processed{extension}"

print("Saving new dataset... ", end="")
data.to_csv(filename)
print("done!")
print("New dataset:", filename)


Saving new dataset... done!
New dataset: ./data/data-processed.csv
